# Assignment 2
##  J.T. Flume (jtf576), Narain Mandyam (ntm555), and Chandler Wann (clw4642)


In [2]:
!pip install yfinance

  Created wheel for yfinance: filename=yfinance-0.1.64-py2.py3-none-any.whl size=24095 sha256=5ac78bf104118c1ed6eac54d8a174ef24ffa8f620a365a85e259ebeef651c8c0
  Stored in directory: c:\users\jtflu\appdata\local\pip\cache\wheels\f9\e9\7e\8b13db3bf3aeb5049d759e10702736fb96753089ac950fddc0
  Created wheel for multitasking: filename=multitasking-0.0.9-py3-none-any.whl size=8368 sha256=d8a4b255406b45169cb9cf7ed41b66f6045b237daa1c94fe705a96a9c6e2a113
  Stored in directory: c:\users\jtflu\appdata\local\pip\cache\wheels\57\6d\a3\a39b839cc75274d2acfb1c58bfead2f726c6577fe8c4723f13
Successfully built yfinance multitasking


In [38]:
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm

def query_user():
    comparables = []
    tmp = " "
    granularity_list = ['1d','1mo','3mo']
    stock = ''
    while not stock.isalpha():
    
        stock = input("Choose a stock ticker:")
        print(stock.isalpha())
        if not stock.isalpha(): 
            print("Invalid Input, Try again")



    comp_number = int(input("Enter the number of comparables: "))
    while comp_number > 0:  
        tmp = input("Enter comparable: ")
        if not tmp.isalpha():
            print("Invalid Input, Try again")
        comparables.append(tmp)
        comp_number -= 1

    start_date = input("Choose a start date [yyyy-mm-dd]: ")

    end_date = input("Choose an end date [yyyy-mm-dd]: ")

    granularity = ""
    while granularity not in granularity_list: 
        granularity = input("Enter Granularity in format [1d,1mo,3mo]:" )
        if granularity not in granularity_list:
            print("wrong input, try again.")

    cap_type = 2
    while cap_type not in [0,1]:
        cap_type = int(input("Is there constant debt [0], or constant debt_to_equity ratio[1]?: "))
        if cap_type not in [0,1]:
            print("invalid input, try again." )
    if cap_type > 0: 
        
        debt_beta = float(input("Choose a beta of debt:"))
    else:
        debt_beta = 0 
    return stock,comparables,start_date,end_date,granularity,cap_type,debt_beta

def get_returns(SYMB, start_date ='2016-10-21', end_date = '2021-10-21',gran = "1d" ):

    target = yf.download(tickers = SYMB,start = start_date, end= end_date, interval = gran ,progress =False).dropna() 
    target = target[['Adj Close']]
    target['ret'] = (target['Adj Close'] - target['Adj Close'].shift())/target['Adj Close'].shift() 

    return target.iloc[1:]

def get_beta(SYMB,gran = '1d'): 
    target = get_returns(SYMB, gran = gran)
    market = get_returns("SPY",gran = gran)
    col_names = [SYMB,"SPY"]

    comb = pd.concat([target['ret'],market['ret']],axis = 1)
    comb.columns = col_names
    ols = sm.formula.ols(SYMB + " ~ SPY",data = comb)
    res = ols.fit()
    return res.params[1]


def get_balance_sheet(SYMB):
    ticker = yf.Ticker(SYMB)
    return ticker.balance_sheet

def get_tax_rate(SYMB):
    ticker = yf.Ticker(SYMB)
    financials = ticker.financials
    ebt = financials.loc['Ebit']  + financials.loc["Interest Expense"]
    tax_rate = financials.loc['Income Tax Expense']/ ebt
    return np.mean(tax_rate[:3])

def get_DE_ratio(SYMB):
    ticker = yf.Ticker(SYMB)
    return ticker.info['debtToEquity']/100

# def get_DE_ratio(SYMB):
#     #ticker = yf.Ticker(SYMB)
#     bs = get_balance_sheet(SYMB)
#     print(bs)
#     return bs.loc['Long Term Debt'][0]/bs.loc['Total Stockholder Equity'][0]

def create_table(tickers,beta_d  = .3,start_date ='2016-10-21', end_date = '2021-10-21', gran = '1d'):
    table  = pd.DataFrame(columns = ['symbol','beta_e','beta_d','debt/equity','tax_rate'])
    for ticker in tickers:
        beta_e = get_beta(ticker)
        d_to_e = get_DE_ratio(ticker)
        tax_rate = get_tax_rate(ticker)
        table = table.append({"symbol":ticker,'beta_e':beta_e,'beta_d':beta_d,'debt/equity':d_to_e,'tax_rate':tax_rate},ignore_index = True)
    return table
def unlever(table,cap_type = 0):
    if cap_type == 0:
        beta_u = table['beta_e']/(1+(1-table['tax_rate'])*table['debt/equity'])
        table['beta_u'] = beta_u
    elif cap_type == 1:
        beta_u = table['beta_e']/(1+table['debt/equity'])+table['beta_d']*table['debt/equity']/(1+table['debt/equity'])
        table['beta_u'] = beta_u
        
    #TODO other types ( constant D/E, beta_d != 0)
    return np.mean(beta_u)
    
def relever(d_to_e,beta_ua,tax_rate,cap_type = 0):
    if cap_type == 0:
        beta_e = beta_ua*(1+(1-tax_rate)*d_to_e)
    elif cap_type == 1:
        beta_e = beta_ua*(1+d_to_e) - table['beta_d']*(d_to_e)
    #TODO other types ( constant D/E, beta_d != 0)
    return beta_e

    

In [71]:
get_returns("AAPL")

,Adj Close,ret
Date,,
2016-10-24,27.544031,0.009005
2016-10-25,27.684504,0.005100
2016-10-26,27.061749,-0.022495
2016-10-27,26.801878,-0.009603
2016-10-28,26.623947,-0.006639
...,...,...
2021-10-14,143.550491,0.020226
2021-10-15,144.628922,0.007513
2021-10-18,146.336426,0.011806


In [72]:
# bs = get_balance_sheet("AAPL")
# bs2 = get_balance_sheet("MSFT")
# bs3 = get_balance_sheet("GOOGL")
# de = get_DE_ratio("MSFT")

In [73]:
# bs.loc['Long Term Debt'][0]/bs.loc['Total Stockholder Equity'][0]

In [74]:
#bs2.loc['Long Term Debt'][0]/bs2.loc['Total Stockholder Equity'][0]

In [75]:
#bs3.loc['Long Term Debt'][0]/bs3.loc['Total Stockholder Equity'][0]

In [76]:
beta_yf = yf.Ticker("AAPL").info['beta']
beta_yf

1.205714

In [77]:
get_tax_rate("AAPL")

0.1543197575959034

In [78]:
beta_custom = get_beta("AAPL")

In [79]:
np.abs(beta_custom - beta_yf)/beta_custom

0.020009259162590255

In [17]:
table = create_table(['GOOGL','MSFT'])

In [18]:
table

,symbol,beta_e,beta_d,debt/equity,tax_rate
0,GOOGL,1.107316,0.3,0.057661,0.154643
1,MSFT,1.213571,0.3,0.352664,0.143029


In [93]:
b_ua

0.993886585466758

In [94]:
aapl_tax_rate = get_tax_rate("AAPL")
aapl_de = get_DE_ratio("AAPL")

In [95]:
aapl_de

1.729370740212395

In [96]:
relever(aapl_de,b_ua,aapl_tax_rate)

2.447440416105389

In [6]:
get_beta("AAPL") # BIG ERROR

1.2303313252946502

In [97]:
b_ua2 = unlever(table,cap_type = 1)
b_ua2

1.0193452718785656

In [98]:
relever(aapl_de,b_ua,aapl_tax_rate,cap_type = 1)

0    2.193874
1    2.193874
Name: beta_d, dtype: float64

In [46]:

def cost_of_capital(equity_beta):
    risk_free = None
    while risk_free == None:
        try:
            market_return = float(input("Market return as a percent:"))/100
            risk_free = float(input("Risk free Rate as percent:"))/100
        except:
            print('Invalid Input, try again!')
    capm = risk_free + (equity_beta*(market_return-risk_free))
    return capm


In [27]:
stock,comparables,start_date,end_date,granularity,cap_type,debt_beta = query_user()

Choose a stock ticker:AAPL
True
Enter the number of comparables: 3
Enter comparable: GOOGL
Enter comparable: AMZN
Enter comparable: MSFT
Choose a start date [yyyy-mm-dd]: 2015-12-31
Choose an end date [yyyy-mm-dd]: 2021-11-05
Enter Granularity in format [1d,1mo,3mo]:1mo
Is there constant debt [0], or constant debt_to_equity ratio[1]?: 0
Choose a beta of debt:.3


In [28]:
tbl = create_table(comparables,debt_beta,start_date,end_date,gran = granularity)
tbl

,symbol,beta_e,beta_d,debt/equity,tax_rate
0,GOOGL,1.107316,0.3,0.11493,0.154643
1,AMZN,0.957294,0.3,1.13001,0.142969
2,MSFT,1.213571,0.3,0.51938,0.143029


In [29]:
b_ua2 = unlever(tbl,cap_type = cap_type)

In [30]:
aapl_tax_rate = get_tax_rate("AAPL")
aapl_de = get_DE_ratio("AAPL")
relever(aapl_de,b_ua2,aapl_tax_rate,cap_type = 0)

2.2030145742863967

In [35]:
def main():
    stock,comparables,start_date,end_date,granularity,cap_type,debt_beta = query_user()
    print("\nCreating Table...")
    tbl = create_table(comparables,debt_beta,start_date,end_date,gran = granularity)
    print("\nHere is the Table of Comparables: ")
    print("\n",tbl)
    if cap_type == 1: 
        stock_debt_beta = input("What is the beta debt of the stock? ")
    b_ua = unlever(tbl,cap_type = cap_type)
    print("\nTable after Unlevering: ")
    print(tbl)
    stock_de = get_DE_ratio(stock)
    stock_tax_rate = get_tax_rate(stock)
    stock_beta_e = relever(stock_de,b_ua,stock_tax_rate,cap_type = 0)
    print(f"\nBeta of stock {stock}: {stock_beta_e}")
    stock_scl = get_beta(stock)
    err = np.abs(stock_beta_e - stock_scl)/stock_beta_e
    print("\nError of comparable method vs SCL: ",err)
    capm_comp = cost_of_capital(stock_beta_e)
    capm_scl = cost_of_capital(stock_scl)
    print("difference in returns[capm of SCL - capm of Comp]: ",(capm_scl-capm_comp))

In [45]:
main()

Choose a stock ticker:AAPL
True
Enter the number of comparables: 3
Enter comparable: AMZN
Enter comparable: GOOGL
Enter comparable: MSFT
Choose a start date [yyyy-mm-dd]: 2015-12-31
Choose an end date [yyyy-mm-dd]: 2021-11-05
Enter Granularity in format [1d,1mo,3mo]:1mo
Is there constant debt [0], or constant debt_to_equity ratio[1]?: 0

Creating Table...

Here is the Table of Comparables: 

   symbol    beta_e beta_d  debt/equity  tax_rate
0   AMZN  0.957294      0      1.13001  0.142969
1  GOOGL  1.107316      0      0.11493  0.154643
2   MSFT  1.213571      0      0.51938  0.143029

Table after Unlevering: 
  symbol    beta_e beta_d  debt/equity  tax_rate    beta_u
0   AMZN  0.957294      0      1.13001  0.142969  0.486318
1  GOOGL  1.107316      0      0.11493  0.154643  1.009260
2   MSFT  1.213571      0      0.51938  0.143029  0.839787

Beta of stock AAPL: 2.168770954756913

Error of comparable method vs SCL:  0.4327057896258921
Market return as a percent:12
Risk free Rate as per

In [ ]:
!jupyter nbconvert --to pdf assignment_2_final.ipynb 